# Вычисление конверсии

Например, рассмотрим продуктовую метрику "конверсия из показа карточки контента в просмотр":  
* пользователь заходит на главную страницу ivi
* рекомендательная система показывает постеры нескольких карточек контента
* если рекомендательная система угадала вкусы пользователя - начинается просмотр контента. Если выдача рекомендаций пользователю не подходит - событие "просмотр контента" не возникает

![main_page_ivi](img/main_page_ivi.png)

Вычислим эту метрику с помощью SQL на основании данных событийной аналитики
* берём только события с платформ `xboxOne` и `Windows 10`
* тип события `page_impression` (показ контента) и `click` (клик по постеру)
* валидация событий: просмотры, у которых отсутствует `impression` из выборки исключаем
* считаем конверсию по датам в разбивке по платформам

Цель исследования - понять, есть ли отличия в конверсии между просмотрами на платформах `xboxOne` и `Windows 10`

In [14]:
import os
import pandas as pd
from sqlalchemy import create_engine


postgres_host = os.environ['APP_POSTGRES_HOST'] # '40.89.151.14'
database = 'postgres'
port = '5432' # '5433'
user = 'postgres'
# user = 'mai'
# password = '1930'

engine = create_engine(
    f'postgresql://{user}:{password}@{postgres_host}:{port}/{database}'
)


sql_str = """
    SELECT
        user_id_for_mai,
        COUNT(*) as num_actions
    FROM movie.events
    GROUP BY user_id_for_mai
    ORDER BY 2 DESC
    LIMIT 100
"""

user_actions_count_df = pd.read_sql(sql_str, engine)
user_actions_count_df.head()

,user_id_for_mai,num_actions
0,5069085339,57269
1,5053362246,38221
2,1330869843,30485
3,5051497926,25133
4,5069470050,24636


In [27]:
# Создаём переменную окружения с вашим именем

os.environ['STUDENT_USERNAME'] = 'adzhumurat'

# Задание 1

Поле `subsite_title` содержит в себе идентификатор, объединяющий платформу и страну, где произошло конкретное событие.

Посчитайте с помощью `GROUP BY` число событий в каждом `subsite_title` и отсортируйте полученный список *по убыванию* счётчика событий. 

In [ ]:
pd.read_sql(
    """
    /* ------------------------ */
    /* --- ВАШ КОД ТУТ -------- */
    SELECT
        COUNT(*) as num_actions
    FROM movie.events
    /* ------------------------ */
    """,
    engine
).head(40)

## Ответьте на несколько вопросов:

**Вопрос 1:** на каких двух `subsite_title` больше всего событий?

In [23]:
print("""
    ВАШ ОТВЕТ
""")


    ВАШ ОТВЕТ



**Вопрос 2**: Сколько всего различных `subsite_title` в наборе даных?

In [28]:
print(f"""
    ВАШ ОТВЕТ
    
    {os.environ['STUDENT_USERNAME']}
""")


    ВАШ ОТВЕТ
    adzhumurat



**Вопрос 3**: На каком `subsite_title` меньше всего событий?

In [29]:
print(f"""
    ВАШ ОТВЕТ
    
    {os.environ['STUDENT_USERNAME']}
""")


    ВАШ ОТВЕТ
    
    adzhumurat



# Задание 2

Примените к выборке фильтрацию

* оставьте события только с двух самых популярных `subsite_title` (из предыдущего задания)
* уберите все events, у которых `content_id <= 0`

Далее после фильтров вычислите количество событий **по дням**, для этого

* превратите `rocket_datetime` в строку вида `YYYY-MM-DD` и назовите новое поле `dt`
* сгруппируйте данные по `dt`
* посчитайте внутри дня сумму событий `actions_count` и отсортируйте выборку по возрастанию поля `dt`
* сохраните выборку в `pandas.DataFrame` с названием `activity_by_dt_df`
* нарисуйте с помощью метода `activity_by_dt_df.plot()` картинку с активностью по дням

Результат должен получится примерно такой:

![actions_count_by_dt](img/actions_count_by_dt.png)

In [37]:
# -------------- ВАШ КОД ТУТ -------------- #




# ----------------------------------------- #

# Задание 3


Нужно сформировать выборку на которой посчитаем конверсию и сохранить запрос в переменную `raw_stat_sql_str`

Для каждой пары пользователь + контент нужно выяснить 2 вещи:
* видел ли пользователь карточку контента
* совершил ли он просмотр данного контента

Для этого

* добавьте в группировку из предыдущего запроса поля `user_id_for_mai`, `content_id`, `subsite_title`
* добавьте аггрегированное поле `MAX(CASE WHEN name in ('page_impression', 'click') THEN 1 ELSE 0 END) as content_impression`
* добавьте аггрегированное поле `MAX(CASE WHEN name='content_watch' THEN 1 ELSE 0 END) as content_watch`

Код ниже выводит общее количество событий в полученной выборке - должно быть `301276` записей.

In [39]:
raw_stat_sql_str = """
    /* ------------------------ */
    /* --- ВАШ КОД ТУТ -------- */
    SELECT
        COUNT(*) as num_actions
    FROM movie.events
    /* ------------------------ */
"""

# Дальше ничего менять не нужно
user_actions_agg_df = pd.read_sql(
    f"""
        SELECT
            COUNT(*) as raw_stat_count
        FROM (
            {raw_stat_sql_str}
        ) as raw_stat
    """,
    engine
)
user_actions_agg_df.head()

,count
0,301276


# Расчёт конверсии

Если всё сделали правильно - код ниже выведет значение конверсии по дням для двух самых популярных `subsite_title` в виде таблички

In [40]:
sql_str = f"""
SELECT
    dt,
    subsite_title,
    SUM(content_watch)::float / SUM(content_impression)::float as conversion
FROM (
    {raw_stat_sql_str}
) as content_watches
WHERE
    content_impression > 0 /* Отфильтруем  "мусорные" случаи, когда */
GROUP BY date, subsite_title
ORDER BY date, subsite_title
"""

conversion_df = pd.read_sql(sql_str, engine)

conversion_df.head()

,date,subsite_title,conversion
0,2019-06-01,Windows 10,0.197802
1,2019-06-01,xboxOne,0.200704
2,2019-06-02,Windows 10,0.287532
3,2019-06-02,xboxOne,0.270677
4,2019-06-03,Windows 10,0.439331


# Задание 4

Постройте график следующего вида
* значение конверсии по дням
* каждый `subsite_title` отдельной линии

Должно получится как-то так

![conversions](img/conversions.png)

Готово! Поздравляю с успешным выполнением домашки